#### Preprocessing Setup
***

In [1]:
import os
curr_dir = os.getcwd()
base_dir = os.path.dirname(curr_dir)
%cd -q $base_dir
!pip install -q -r requirements.txt 
%cd -q $base_dir/core/
# Base directory: /project-noteGen
# Current directory: /core  

import numpy as np
import csv
import pandas as pd
import librosa
from scipy.io.wavfile import read as read_wav
from scipy.io.wavfile import write as write_wav

In [ ]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())
cuda_id = torch.cuda.current_device()
print(cuda_id)
print(torch.cuda.get_device_name(cuda_id))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t = torch.cuda.get_device_properties(device).total_memory * 1e-9
r = torch.cuda.memory_reserved(device) * 1e-9
a = torch.cuda.memory_allocated(device) * 1e-9
print(f"total: {t:.2f} GB")
print(f"reserved: {r:.2f} GB")
print(f"used: {a:.2f} GB")

######
##### Enclosed functions (stored in preprocess.py):
> - construct_note_samples(song_raw, song_note_labels, ds_rate=None)
> - downsample_wav(filepath, destpath, rate)
> - get_folder_size(path)
> - get_dataset_size()

######
#### Investigate and use wav files
***

In [3]:
# Investigate training song 1727.wav with numpy and scipy wav module

a = read_wav(f"{base_dir}/data/raw/train_data/1727.wav") # Reads the wav file

print("Wav File 1727\n")
a_sample_rate = a[0]                        # Sample rate
a_samples = np.array(a[1], dtype=float)     # Store samples (equivalent to a_samples = a[1] for this dataset)
a_samples_shape = np.shape(a_samples)       # Count samples

print(f"Samples per second: {a_sample_rate}\n")

print(f"Number of samples: {a_samples_shape}\n")

print(f"Number of seconds: {a_samples_shape[0] * 1/a_sample_rate:.4f}\n")

rounded_a_samples = np.round(a_samples, decimals=4)
print(f"First 30 samples at t = 10 seconds:\n{rounded_a_samples[10*a_sample_rate:10*a_sample_rate+30]}")


# Create a wav file for a piano note's samples (Referencing 1727.csv)
a_piano_note = a_samples[9182:62430]
print(f"\nExample piano quarter note:\n{np.round(a_piano_note, decimals=5)}")
save_path = f"{base_dir}/data/general"
file_handle = os.path.join(save_path, "example_piano_note.wav")
write_wav(file_handle, a_sample_rate, a_piano_note.astype(np.float32))    # Writes(creates) a wav file for a note

# Create a wav file for a violin note's samples
a_violin_note = a_samples[670686:722910]
print(f"\nExample violin quarter note:\n{np.round(a_violin_note, decimals=5)}")
save_path = f"{base_dir}/data/general"
file_handle = os.path.join(save_path, "example_violin_note.wav")
write_wav(file_handle, a_sample_rate, a_violin_note.astype(np.float32))

# Create a wav file for the final note's samples
a_finale_note = a_samples[19233758:19421150]
print(f"\nExample finale half note:\n{np.round(a_finale_note, decimals=5)}")
save_path = f"{base_dir}/data/general"
file_handle = os.path.join(save_path, "example_finale_note.wav")
write_wav(file_handle, a_sample_rate, a_finale_note.astype(np.float32))

Wav File 1727

Samples per second: 44100

Number of samples: (19715328,)

Number of seconds: 447.0596

First 30 samples at t = 10 seconds:
[ 0.0018 -0.0011 -0.0044 -0.0079 -0.0119 -0.0168 -0.0225 -0.0291 -0.0369
 -0.0453 -0.0535 -0.0618 -0.0694 -0.0762 -0.0826 -0.0875 -0.0909 -0.093
 -0.0934 -0.0928 -0.0919 -0.0894 -0.0861 -0.0816 -0.0754 -0.069  -0.062
 -0.0547 -0.0469 -0.0389]

Example piano quarter note:
[-0.00244 -0.00244 -0.0022  ... -0.00867 -0.00873 -0.00879]

Example violin quarter note:
[-0.00974 -0.01053 -0.01138 ...  0.00049 -0.00027  0.00015]

Example finale half note:
[-0.00983 -0.00977 -0.00958 ...  0.00055  0.00061  0.00052]


######
#### Investigate and use csv files
***

In [4]:
# Investigate training label 1727.csv with numpy, pandas, and csv module

# Read csv file using csv module
b = csv.reader(open(f"{base_dir}/data/raw/train_labels/1727.csv"))
print("\nLOADING #1727 IN CSV MODULE:")
for i, row in enumerate(b):                                               # Print first 5 rows' starts/ends
    #print(row)
    print(row[0], row[1])                                       
    if i > 5: break

# Read csv file using pandas
b_df = pd.read_csv(f"{base_dir}/data/raw/train_labels/1727.csv")
print("\nIN PANDAS:")
display(b_df.head(n=5))                                                   # Display first 5 note labels
print(f"1727's total number of notes: {b_df.shape[0]}")                   # Total number of notes
print(f"Start time of the 5th note: {b_df.at[5, 'start_time']}")          # Access specific column value of note
print(f"------[Note 5]------\n{b_df.loc[5]}")                             # Access specific note


# View in numpy by converting pandas to numpy (***)
b_np = b_df.to_numpy()
print("\nIN NUMPY:")
print(f"\nShape of numpy labels: {np.shape(b_np)}")                       # Shape
print(f"\nNote 5: {b_np[5]}")                                             # Note row
print(f"\nStart time: {b_np[5, 0]}   End time: {b_np[5, 1]}")             # Note elements

# Find average duration of notes
average_time = np.mean((b_np[:, 1] - b_np[:, 0]) / 44100)
print(f"\nThe average duration of 1727's notes is {average_time:.5f} seconds")

# Obtain samples for specific note, and generate a wav file
a_note_456 = a_samples[b_np[455, 0]:b_np[455, 1]]                                # Access in numpy #(0+455)
# a_note_456 = a_samples[b_df.at[455, 'start_time']:b_df.at[455, 'end_time']]    # Alternative in pandas
print(f"\nThe 456th Note's Samples:\n{np.round(a_note_456, decimals=5)}")
save_path = f"{base_dir}/data/general"
file_handle = os.path.join(save_path, "example_456th_note.wav")
write_wav(file_handle, 44100, a_note_456.astype(np.float32))


LOADING #1727 IN CSV MODULE:
start_time end_time
9182 90078
9182 33758
9182 62430
9182 202206
9182 62430
33758 62430

IN PANDAS:


,start_time,end_time,instrument,note,start_beat,end_beat,note_value
0,9182,90078,43,53,4.0,1.5,Dotted Quarter
1,9182,33758,42,65,4.0,0.5,Eighth
2,9182,62430,1,69,4.0,1.0,Quarter
3,9182,202206,44,41,4.0,3.5,Whole
4,9182,62430,1,81,4.0,1.0,Quarter


1727's total number of notes: 6580
Start time of the 5th note: 33758
------[Note 5]------
start_time     33758
end_time       62430
instrument        42
note              60
start_beat       4.5
end_beat         0.5
note_value    Eighth
Name: 5, dtype: object

IN NUMPY:

Shape of numpy labels: (6580, 7)

Note 5: [33758 62430 42 60 4.5 0.5 'Eighth']

Start time: 33758   End time: 62430

The average duration of 1727's notes is 0.33706 seconds

The 456th Note's Samples:
[0.02539 0.02591 0.02695 ... 0.01593 0.01532 0.0144 ]


######
### Construct the Note Samples for A Song
***

In [5]:
# This function takes the read wav and csv files for a song and constructs a list of note samples,
# where each element of the list contains the numpy samples corresponding to a note in the song

def construct_note_samples(song_raw, song_note_labels, ds_rate=None):
    """
    inputs:
        song_raw: a read wav output for a song returned by read_wav
                    ie song_raw = read_wav(path_to_wav_file)
        song_note_labels: a csv table for a song's notes stored as a numpy array
                    ie song_note_labels = pd.read_csv(path_to_csv_file).to_numpy()
        ds_rate: sampling rate if raw song has been downsampled (ie 22050), default assumes it has not
    output:
        song_note_samples: a numpy array of uneven numpy array objects, each of which contain the
                           samples (from song_raw) pertaining to one note (from song_note_labels)
    """
    if ds_rate is None:
        time_factor = 1
    else:
        time_factor = ds_rate / 44100
    song_raw_samples = song_raw[1]
    song_note_timings = song_note_labels[:, 0:2] * time_factor

    notes_count = np.shape(song_note_timings)[0]
    
    song_note_samples = [[] for i in range(notes_count)]              # The note samples to be built

    for note_i in range(notes_count):
        start_time = int(song_note_timings[note_i, 0])
        end_time = int(song_note_timings[note_i, 1])
        song_note_samples[note_i] = song_raw_samples[start_time:end_time]

    song_note_samples = np.array(song_note_samples, dtype=object)     # Store as np.array of uneven np.arrays
    
    return song_note_samples

In [6]:
# Construct Note Samples for Song #1727 (1727.wav + 1727.csv)

song_raw = read_wav(f"{base_dir}/data/raw/train_data/1727.wav")
song_note_labels = pd.read_csv(f"{base_dir}/data/raw/train_labels/1727.csv").to_numpy()
print(f"Consolidating notes samples for {np.shape(song_note_labels)[0]} notes")

song_note_samples = construct_note_samples(song_raw, song_note_labels)


# Assert note samples are correct
assert np.shape(song_note_samples[2]) == np.shape(a_piano_note)
assert np.shape(song_note_samples[121]) == np.shape(a_violin_note)
assert np.shape(song_note_samples[455]) == np.shape(a_note_456)
assert np.shape(song_note_samples[6579]) == np.shape(a_finale_note)
assert np.allclose(song_note_samples[2], a_piano_note)
assert np.allclose(song_note_samples[121], a_violin_note)
assert np.allclose(song_note_samples[455], a_note_456)
assert np.allclose(song_note_samples[6579], a_finale_note)

# Create a wav file for the 100th note
save_path = f"{base_dir}/data/general"
file_handle = os.path.join(save_path, "example_100th_note.wav")
write_wav(file_handle, 44100, song_note_samples[99].astype(np.float32))
print(f"\nLabels of the 100th note: {song_note_labels[99]}")

# Save note samples in .npy format
np.save(f"{base_dir}/data/general/note_samples_1727.npy", song_note_samples, allow_pickle=True)

# Load note samples from .npy format
loaded_note_samples = np.load(f"{base_dir}/data/general/note_samples_1727.npy", allow_pickle=True)

assert np.allclose(song_note_samples[500], loaded_note_samples[500])
assert np.allclose(song_note_samples[1000], loaded_note_samples[1000])
assert np.allclose(song_note_samples[1500], loaded_note_samples[1500])

Consolidating notes samples for 6580 notes

Labels of the 100th note: [525278 555998 42 60 13.5 0.5 'Eighth']


######
### Construct Note Samples for All Test Songs (Requires 0.62 GB)

In [7]:
# Construct note sample files for each song in test data

test_songs_ids = os.listdir(f"{base_dir}/data/raw/test_data/")

for i, filename in enumerate(test_songs_ids):
    test_songs_ids[i] = int(filename[:4])                      # Gets rid of ".wav"
print(f"Test song ids:  {test_songs_ids}")

for test_song in test_songs_ids:
    test_song_raw = read_wav(f"{base_dir}/data/raw/test_data/{test_song}.wav")
    test_song_note_labels = pd.read_csv(f"{base_dir}/data/raw/test_labels/{test_song}.csv").to_numpy()
    test_song_note_samples = construct_note_samples(test_song_raw, test_song_note_labels)
    np.save(f"{base_dir}/data/numpy/test_note_samples/note_samples_{test_song}.npy", \
            test_song_note_samples, allow_pickle=True)

Test song ids:  [1759, 1819, 2106, 2191, 2298, 2303, 2382, 2416, 2556, 2628]


In [8]:
# Look at test note samples and see if they are correct


note_samples_2298 = np.load(f"{base_dir}/data/numpy/test_note_samples/note_samples_2298.npy", \
                            allow_pickle=True)
song_labels_2298 = pd.read_csv(f"{base_dir}/data/raw/test_labels/2298.csv").to_numpy()
song_raw_2298 = read_wav(f"{base_dir}/data/raw/test_data/2298.wav")
song_samples_2298 = song_raw_2298[1]

assert np.shape(note_samples_2298)[0] == np.shape(song_labels_2298)[0]

# Looking at the 132nd (half) note
trial_note_132 = note_samples_2298[131]
actual_note_132 = song_samples_2298[song_labels_2298[131, 0]:song_labels_2298[131, 1]]
assert np.shape(trial_note_132) == np.shape(actual_note_132)
assert np.allclose(trial_note_132, actual_note_132)

# Create a wav file for the 132nd (half) note
write_wav(f"{base_dir}/data/general/test_note_132_2298.wav", 44100, trial_note_132.astype(np.float32))

######
### Construct Note Samples for All Train Songs** (Requires 68 GB)

In [9]:
# Construct note sample files for each song in train data

train_songs_ids = os.listdir(f"{base_dir}/data/raw/train_data/")

for i, filename in enumerate(train_songs_ids):
    train_songs_ids[i] = int(filename[:4])
print(f"Number of train songs:   {len(train_songs_ids)}")\

""" (Uncomment to Run)
for train_song in train_songs_ids:
    train_song_raw = read_wav(f"{base_dir}/data/raw/train_data/{train_song}.wav")
    train_song_note_labels = pd.read_csv(f"{base_dir}/data/raw/train_labels/{train_song}.csv").to_numpy()
    train_song_note_samples = construct_note_samples(train_song_raw, train_song_note_labels)
    np.save(f"{base_dir}/data/numpy/train_note_samples/note_samples_{train_song}.npy", \
            train_song_note_samples, allow_pickle=True)
"""

Number of train songs:   320


' (Uncomment to Run)\nfor train_song in train_songs_ids:\n    train_song_raw = read_wav(f"{base_dir}/data/raw/train_data/{train_song}.wav")\n    train_song_note_labels = pd.read_csv(f"{base_dir}/data/raw/train_labels/{train_song}.csv").to_numpy()\n    train_song_note_samples = construct_note_samples(train_song_raw, train_song_note_labels)\n    np.save(f"{base_dir}/data/numpy/train_note_samples/note_samples_{train_song}.npy",             train_song_note_samples, allow_pickle=True)\n'

***
#### Size of datasets

In [10]:
# Gets the size of the files in one directory
def get_folder_size(path):
    """
    inputs:
        path: the path to the directory for which get the size
    outputs:
        size_gb: size of files in the directory in gigabytes
    """
    size_b = 0
    for filename in os.listdir(path):
        size_b += os.path.getsize(f"{path}/{filename}")
    size_gb = size_b * 10**(-9)
    
    return size_gb

In [11]:
# Prints dataset sizes for various directories
def get_dataset_size():
    test_wav_songs_size = get_folder_size(f"{base_dir}/data/raw/test_data/")
    train_wav_songs_size = get_folder_size(f"{base_dir}/data/raw/train_data/")

    test_csv_songs_size = get_folder_size(f"{base_dir}/data/raw/test_labels/")
    train_csv_songs_size = get_folder_size(f"{base_dir}/data/raw/train_labels/")

    test_note_samples_size = get_folder_size(f"{base_dir}/data/numpy/test_note_samples/")
    train_note_samples_size = get_folder_size(f"{base_dir}/data/numpy/train_note_samples/")

    dataset_size = test_wav_songs_size + train_wav_songs_size + test_csv_songs_size + \
                    train_csv_songs_size + test_note_samples_size + train_note_samples_size
    print(f"test raw data:  {test_wav_songs_size:.4f} GB")
    print(f"train raw data:  {train_wav_songs_size:.4f} GB")
    print(f"test labels:  {test_csv_songs_size:.4f} GB")
    print(f"train labels:  {test_csv_songs_size:.4f} GB")
    print()
    print(f"test note samples:  {test_note_samples_size:.4f} GB")
    print(f"train note samples:  {train_note_samples_size:.4f} GB")
    print()
    print(f"dataset current total:  {dataset_size:.4f} GB")

In [12]:
get_dataset_size()

test raw data:  0.2613 GB
train raw data:  21.4128 GB
test labels:  0.0006 GB
train labels:  0.0006 GB

test note samples:  0.6237 GB
train note samples:  0.0000 GB

dataset current total:  22.3526 GB


####
#### Accounting for size:
The size of the constructed note samples are on average 3x the size of the songs themselves, perhaps indicating at each moment in a song, 3 notes co-occur on average. However, this varies widely by the composition category: solo, trio, quintent, quartet, sextet...

For a ~20 GB dataset of raw songs and csv labels, transforming all songs into note samples will produce ~60-70 GB.

The dataset size of note samples can be reduced by 1) Choosing a select number of songs to train, 2) Choosing a select number of notes per song to construct, and 3) Downsampling the songs.

Downsampling reduces the size of the raw songs, which in turn reduces the size of the constructed note samples. Downsampling is performed by sampling the original file at a lower sampling rate, and storing the result by its new sample rate.

Downsampling also requires adjusting the timestep columns in train_labels and test_labels, which assume 44100kHz sampling. The function "construct_note_samples" performs the adjustment by taking an optional downsampling rate and calculating a time factor.

***
### Downsampling a song with Librosa and Scipy

In [13]:
# How to Downsample:
# 1) Load in from librosa in desired sample rate        Sample wav file at that rate
# 2) Write out with scipy in matching sample rate       Expand samples by that rate

def downsample_wav(filepath, destpath, rate):
    """
    inputs:
        filepath: path to the wav file to be downsampled
        destpath: path and name to store the downsampled wav file
        rate: the downsampling rate
    outputs:
        downsampled_raw: the read wav output for the downsampled file
        downsampled_size: the size of the downsampled file
    """
    rate = int(rate)
    y = librosa.load(filepath, sr=rate)[0]
    write_wav(destpath, rate=rate, data=y)
    downsampled_raw = read_wav(destpath)
    downsampled_size = os.path.getsize(destpath)
    return downsampled_raw, downsampled_size

######
#### Quality and Size of downsampled songs

In [14]:
# Downsampling song 1727 (Piano Quintet)
"""
path_1727 = f"{base_dir}/data/raw/train_data/1727.wav"
dest_dir = f"{base_dir}/data/downsampling_comparisons/"

y_1x, size_1x = downsample_wav(path_1727, f"{dest_dir}/1727_downsampled_0x.wav", 44100)
y_2x, size_2x = downsample_wav(path_1727, f"{dest_dir}/1727_downsampled_2x.wav", 44100 / 2)
y_4x, size_4x = downsample_wav(path_1727, f"{dest_dir}/1727_downsampled_4x.wav", 44100 / 4)

print(f"Original size:         {size_1x*10**-6:.3f} MB")
print(f"Downsampled x2 size:   {size_2x*10**-6:.3f} MB")           # Minor difference in audio
print(f"Downsampled x4 size:   {size_4x*10**-6:.3f} MB")           # Sizable difference in audio
"""
"""
Original size:         78.861 MB
Downsampled x2 size:   39.431 MB
Downsampled x4 size:   19.715 MB
"""

'\nOriginal size:         78.861 MB\nDownsampled x2 size:   39.431 MB\nDownsampled x4 size:   19.715 MB\n'

In [15]:
# Downsampling song 2208 (Solo Piano)
path_2208 = f"{base_dir}/data/raw/train_data/2208.wav"
dest_dir = f"{base_dir}/data/downsampling_comparisons/"

y_1x, size_1x = downsample_wav(path_2208, f"{dest_dir}/2208_downsampled_0x.wav", 44100)
y_2x, size_2x = downsample_wav(path_2208, f"{dest_dir}/2208_downsampled_2x.wav", 44100 / 2)
y_3x, size_3x = downsample_wav(path_2208, f"{dest_dir}/2208_downsampled_3x.wav", 44100 / 3)

print(f"Original size:         {size_1x*10**-6:.3f} MB")
print(f"Downsampled x2 size:   {size_2x*10**-6:.3f} MB")           # Minimal difference in audio          
print(f"Downsampled x3 size:   {size_3x*10**-6:.3f} MB")           # Minor difference in audio

Original size:         21.307 MB
Downsampled x2 size:   10.654 MB
Downsampled x3 size:   7.103 MB


In [16]:
# Downsampling song 2244 (Solo Violin)
path_2244 = f"{base_dir}/data/raw/train_data/2244.wav"

y_1x, size_1x = downsample_wav(path_2244, f"{dest_dir}/2244_downsampled_0x.wav", 44100)
y_2x, size_2x = downsample_wav(path_2244, f"{dest_dir}/2244_downsampled_2x.wav", 44100 / 2)
y_3x, size_3x = downsample_wav(path_2244, f"{dest_dir}/2244_downsampled_3x.wav", 44100 / 3)

print(f"Original size:         {size_1x*10**-6:.3f} MB")
print(f"Downsampled x2 size:   {size_2x*10**-6:.3f} MB")           # Moderate difference in audio       
print(f"Downsampled x3 size:   {size_3x*10**-6:.3f} MB")           # Large difference in audio

Original size:         37.813 MB
Downsampled x2 size:   18.907 MB
Downsampled x3 size:   12.604 MB


######
#### How does downsampling affect constructed note samples

In [17]:
# Comparing size of downsampled note samples for 2244

labels_2244 = pd.read_csv(f"{base_dir}/data/raw/train_labels/2244.csv").to_numpy()

note_samples_2244_0x = construct_note_samples(y_1x, labels_2244)
note_samples_2244_2x = construct_note_samples(y_2x, labels_2244, ds_rate=44100 / 2)
note_samples_2244_3x = construct_note_samples(y_3x, labels_2244, ds_rate=44100 / 3)
np.save(f"{dest_dir}/note_samples_2244_0x.npy", note_samples_2244_0x, allow_pickle=True)
np.save(f"{dest_dir}/note_samples_2244_2x.npy", note_samples_2244_2x, allow_pickle=True)
np.save(f"{dest_dir}/note_samples_2244_3x.npy", note_samples_2244_3x, allow_pickle=True)

size_1x = os.path.getsize(f"{dest_dir}/note_samples_2244_0x.npy")
size_2x = os.path.getsize(f"{dest_dir}/note_samples_2244_2x.npy")
size_3x = os.path.getsize(f"{dest_dir}/note_samples_2244_3x.npy")

print(f"Original note samples:         {size_1x*10**-6:.3f} MB")
print(f"Downsampled x2 note samples:   {size_2x*10**-6:.3f} MB")                   
print(f"Downsampled x3 note samples:   {size_3x*10**-6:.3f} MB")

print("\nSize of the note samples depend on the number of notes : the more overlapping notes, the larger")
print("    For solos like 2244,     size(note_samples.npy) ≈ size(raw_song.wav)")
print("    For quintets like 1727,  size(note_samples.npy) >> size(raw_song.wav)")

Original note samples:         38.705 MB
Downsampled x2 note samples:   19.399 MB
Downsampled x3 note samples:   12.964 MB

Size of the note samples depend on the number of notes : the more overlapping notes, the larger
    For solos like 2244,     size(note_samples.npy) ≈ size(raw_song.wav)
    For quintets like 1727,  size(note_samples.npy) >> size(raw_song.wav)


In [18]:
# Compare quality of individual downsampled notes for 2244

write_wav(f"{dest_dir}/2244_single_high_note_0x.wav", 44100, note_samples_2244_0x[48])
write_wav(f"{dest_dir}/2244_single_high_note_2x.wav", int(44100 / 2), note_samples_2244_2x[48])
write_wav(f"{dest_dir}/2244_single_high_note_3x.wav", int(44100 / 3), note_samples_2244_3x[48])

# Sound virtually identical on a per note basis despite difference in samples:
print(note_samples_2244_0x[48].size, "samples")
print(note_samples_2244_2x[48].size, "samples")
print(note_samples_2244_3x[48].size, "samples")

5120 samples
2560 samples
1707 samples


#####
#### Average number of samples and storage size per note

In [19]:
total_samples = 0
total_notes = 0

all_test_label_names = os.listdir(f"{base_dir}/data/raw/test_labels")
all_train_label_names = os.listdir(f"{base_dir}/data/raw/train_labels")

pitches = [np.inf, -np.inf]

for test_label_name in all_test_label_names:
    if test_label_name.endswith(".csv"):
        test_label = pd.read_csv(f"{base_dir}/data/raw/test_labels/{test_label_name}").to_numpy()
        total_samples += np.sum(test_label[:, 1] - test_label[:, 0])
        total_notes += np.shape(test_label)[0]
        if np.min(test_label[:, 3]) < pitches[0]: pitches[0] = np.min(test_label[:, 3])
        if np.max(test_label[:, 3]) > pitches[1]: pitches[1] = np.max(test_label[:, 3])

for train_label_name in all_train_label_names:
    if train_label_name.endswith(".csv"):
        train_label = pd.read_csv(f"{base_dir}/data/raw/train_labels/{train_label_name}").to_numpy()
        total_samples += np.sum(train_label[:, 1] - train_label[:, 0])
        total_notes += np.shape(train_label)[0]
        if np.min(train_label[:, 3]) < pitches[0]: pitches[0] = np.min(train_label[:, 3])
        if np.max(train_label[:, 3]) > pitches[1]: pitches[1] = np.max(train_label[:, 3])
    
avg_samples_per_note = total_samples / total_notes
avg_size_per_note = avg_samples_per_note * 4 * 10**-6                 # 4 bytes per sample (float32)
print(f"Average samples per note: {avg_samples_per_note:.0f}")
print(f"Average size of one note: {avg_size_per_note:.3f} MB")

print(f"\nRange of pitches: {pitches}")

Average samples per note: 15659
Average size of one note: 0.063 MB

Range of pitches: [21, 104]
